# Import Libraries and Verify Earth Engine Works

In [61]:
import ee
import geopy
from geopy.geocoders import Nominatim
from PIL import Image
import requests
from io import BytesIO
import math
ee.Initialize(project='gthack25')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


# Get City Images

In [63]:
def get_city_coordinates(city_name):
    # INPUTS: CITY NAME
    # OUTPUT: LAT AND LONG
    geolocator = Nominatim(user_agent="city_locator")
    location = geolocator.geocode(city_name)
    return location.latitude, location.longitude

# Function to get satellite image of a specific region using USDA/NAIP/DOQQ dataset
def retrieve_naip_image(lat, long, date2, deltalr=0.003625, deltaud=0.003625, date1='2018-01-01'):
    """
    INPUTS: 
        LATITUDE
        LONGITUDE
        DATE2 - END DATE
        DATE1 - START DATE
        DELTALR - HALFWIDTH OF REGION LEFT TO RIGHT IN DEGREES (STRETCH THESE SO THE RADIUS IS A .5 MILE OR A MILE)
        DELTAUD - HALFWIDTH OF REGION TOP TO BOTTOM IN DEGREES
    OUTPUTS: IMAGE DATA
    """
    #CREATE A BOUNDING BOX FOR IMAGE (WOULD PROBABLY MESS WITH THIS TO ENFORCE A SHAPE)
    region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
    #SLICE IMAGE CLOSEST OT DATE 1 AND AND WITHIN SHAPE OF BOUNDING BOX
    dataset = ee.ImageCollection('USDA/NAIP/DOQQ') \
        .filter(ee.Filter.date(date1, date2)) \
        .filter(ee.Filter.bounds(region))
    return dataset

def save_image_from_url(url, filename):
    #SAVE IMAGE DATA TO PNG
    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save(filename)
        print(f"Satellite image saved as {filename}")
    else:
        print("Error fetching the satellite image.")

#Function to get satellite image of a city
def get_satellite_image(city_name, range_miles, date1='2018-01-01', date2='2025-02-22'):
    # Convert range from miles to meters (1 mile = 1609.34 meters)
    range_meters = range_miles * 1609.34
    
    # Get coordinates of the city
    latitude, longitude = get_city_coordinates(city_name)
    
    # Define the region of interest
    point = ee.Geometry.Point([longitude, latitude])
    region = point.buffer(range_meters).bounds()
    
    # Retrieve the satellite image from USDA/NAIP/DOQQ dataset
    dataset = retrieve_naip_image(latitude, longitude, date2, date1=date1)
    image = dataset.sort('system:time_start', False).first().clip(region)
    
    # Get the URL for the image
    url = image.getThumbURL({
        'min': 0,
        'max': 255,
        'dimensions': '1024x1024',
        'region': region,
        'format': 'png'
    })
    
    # Fetch and save the image
    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save("satellite_image.png")
        print("Satellite image saved as satellite_image.png")
    else:
        print("Error fetching the satellite image.")
    
    return image


# # Function to get and save multiple satellite images of a city area
# def get_multiple_satellite_images(city_name, date1='2018-01-01', date2='2025-02-22', step_miles=1):
#     # Convert step size from miles to degrees (approximation: 1 mile ≈ 0.0145 degrees)
#     step_degrees = step_miles * 0.0145
    
#     # Get coordinates of the city
#     latitude, longitude = get_city_coordinates(city_name)
    
#     # Define the range for sliding over the area (5 miles in each direction from the center)
#     range_miles = 15
#     range_degrees = range_miles * 0.0145
    
#     # Loop over the area and take multiple images
#     num_steps = math.ceil(range_degrees / step_degrees)
#     for i in range(-num_steps, num_steps + 1):
#         for j in range(-num_steps, num_steps + 1):
#             lat = latitude + i * step_degrees
#             long = longitude + j * step_degrees
#             dataset = retrieve_naip_image(lat, long, date2, date1=date1)
#             image = dataset.sort('system:time_start', False).first().clip(ee.Geometry.Point([long, lat]).buffer(1609.34))
#             url = image.getThumbURL({
#                 'min': 0,
#                 'max': 255,
#                 'dimensions': '1024x1024',
#                 'region': ee.Geometry.Point([long, lat]).buffer(1609.34).bounds(),
#                 'format': 'png'
#             })
#             filename = f"CLIPPED_DATA/satellite_image_{i}_{j}.png"
#             save_image_from_url(url, filename)

In [66]:
city_name = "Charleston, SC"
range_miles = 1

# Get the satellite image
satellite_image = get_satellite_image(city_name, range_miles)

# Display the image (for visualization purposes)
satellite_image.show()

Satellite image saved as satellite_image.png


In [53]:
# # Define the city
# city_name = "Tulsa, OK"

# # Get and save multiple satellite images
# get_multiple_satellite_images(city_name)

Satellite image saved as CLIPPED_DATA/satellite_image_-15_-15.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-14.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-13.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-12.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-11.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-10.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-9.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-8.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-7.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-6.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-5.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-4.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-3.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-2.png
Satellite image saved as CLIPPED_DATA/satellite_image_-15_-1.png
Satellite image sav

KeyboardInterrupt: 